In [ ]:
import functools
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
TEST_FILE = "ibmML_Test.csv"
np.set_printoptions(precision=3, suppress=True)

In [ ]:
read_csv = pd.read_csv(TEST_FILE)

In [ ]:
read_csv.dtypes

In [ ]:
read_csv['notification_time'] = pd.to_datetime(read_csv['notification_time'] , format="%H:%M:%S")

In [ ]:
def convert(time):
    val = 3600*time.hour + 60*time.minute + time.second
    return val

In [ ]:
data = {'notification_time': []}
for index,value in read_csv['notification_time'].items():
    val = convert(value)
    data['notification_time'].append(val)  

In [ ]:
xs = read_csv.pop('notification_time')
read_csv.columns

In [ ]:
read_csv[ "notification_time"] = data['notification_time'] 

In [ ]:
read_csv['clicked'] = pd.Categorical(read_csv['clicked'])
read_csv['clicked'] = read_csv.clicked.cat.codes
read_csv.dtypes

In [ ]:
read_csv.columns
if 'clicked' in read_csv:
    print("exists")
read_csv.plot(x = "notification_time" , y='clicked' , kind='scatter')

In [ ]:
target = read_csv.pop('clicked')


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((read_csv.values, target.values))
test_data = dataset.take(5)

In [ ]:
for feat, targ in dataset.take(5):
    print('Features: {}, Target: {}'.format(feat, targ))

In [ ]:
train_dataset = dataset.shuffle(len(read_csv)).batch(1)

In [ ]:
def get_compiled_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu' ,  input_shape=[len(train_dataset)]),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

In [ ]:
model = get_compiled_model()

In [ ]:
model.fit(train_dataset ,epochs = 6)